In [3]:
import os
import boto3
from pyspark.sql import SparkSession


In [4]:
# Configuración de AWS y PySpark
AWS_REGION = "eu-south-2"
BUCKET_NAME_INPUT = "crypto-historical-data-bucket"  # Bucket de entrada (capa bronze)
BUCKET_NAME_OUTPUT = "crypto-silver-data-bucket"  # Bucket de salida (capa silver)



In [5]:
# Configuración de la sesión de Spark
spark = SparkSession.builder \
    .appName("Convert CSV to Parquet") \
    .config("spark.hadoop.fs.s3a.endpoint", f"s3.{AWS_REGION}.amazonaws.com") \
    .getOrCreate()


25/02/27 18:32:05 WARN Utils: Your hostname, MacBook-Air-de-Santiago-2.local resolves to a loopback address: 127.0.0.1; using 10.120.149.127 instead (on interface en0)
25/02/27 18:32:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 18:32:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Lista de criptomonedas
cryptos = {
    "Bitcoin": "BTCUSD",  
    "Ethereum": "ETHUSD", 
    "Ripple": "XRPUSD",  
    "Solana": "SOLUSD",  
    "Dogecoin": "DOGEUSD",
    "Cardano": "ADAUSD",  
    "Shiba Inu": "SHIBUSD", 
    "Polkadot": "DOTUSD",  
    "Aave": "AAVEUSD",  
    "Stellar": "XLMUSD", 
}

In [7]:
# Función para convertir CSV a Parquet y cargarlo a S3
def convert_csv_to_parquet():
    s3_client = boto3.client("s3")

    # Obtener la lista de archivos CSV desde el bucket de entrada
    for crypto_name, symbol in cryptos.items():
        # Definir el prefijo y la ruta del archivo CSV
        crypto_folder = symbol.removesuffix('USD')  # Sin "USD"
        file_prefix = f"{crypto_folder}/"

        # Listar objetos dentro del prefijo (carpeta del símbolo)
        response = s3_client.list_objects_v2(Bucket=BUCKET_NAME_INPUT, Prefix=file_prefix)

        if 'Contents' in response:
            for obj in response['Contents']:
                file_key = obj['Key']
                # Leer archivo CSV desde S3 con PySpark
                csv_file_path = f"s3a://{BUCKET_NAME_INPUT}/{file_key}"
                print(f"Convirtiendo {file_key} de CSV a Parquet...")

                # Leer el archivo CSV
                df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

                # Definir la ruta de salida para el archivo Parquet
                parquet_file_path = f"s3a://{BUCKET_NAME_OUTPUT}/{file_key.replace('.csv', '.parquet')}"

                # Guardar el DataFrame como Parquet
                df.write.parquet(parquet_file_path, mode="overwrite")
                print(f"Archivo convertido y guardado en: {parquet_file_path}")

        else:
            print(f"No se encontraron archivos para {crypto_name} en el bucket.")


In [8]:
if __name__ == "__main__":
    convert_csv_to_parquet()

Convirtiendo BTC/2021/Bitcoin_historical_data_2021.csv de CSV a Parquet...


25/02/27 18:32:07 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://crypto-historical-data-bucket/BTC/2021/Bitcoin_historical_data_2021.csv.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.s

Py4JJavaError: An error occurred while calling o29.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Dat